<a href="https://colab.research.google.com/github/Anjasfedo/Learning-TensorFlow/blob/main/eat_tensorflow2_in_30_days/Chapter2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2-3 Automatic Differentiate

The neural network relies on back propagations to calculate gradients and update parameters in the network. Gradient calculatio is complicated which is easy to occur mistakes.

The framework for deep learning helps to calculate gradient automatically.

`tf.GradientTape` is usually used to record forward calculation in TensorFlow, and reverse this "tape" to obtain the gradient.

This is the automatic differentiate in TensorFlow.

## 1. Calculate the Derivative Using the Gradient Tape

In [1]:
import tensorflow as tf
import numpy as np

# Calculate the derivative of f(x) = a*x**2 + b*x + c

x = tf.Variable(0.0, name='x', dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(2.0)
c = tf.constant(3.0)

with tf.GradientTape() as tape:
    y = a * tf.pow(x, 2) + b * x + c

dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(2.0, shape=(), dtype=float32)


In [2]:
# Use watch to calculate derivatives of the constant tensor

with tf.GradientTape() as tape:
  tape.watch([a, b, c])
  y = a * tf.pow(x, 2) + b * x + c

dy_dx, dy_da, dy_db, dy_dc = tape.gradient(y, [x, a, b, c])
print(dy_dx)
print(dy_da)
print(dy_db)
print(dy_dc)

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(0.0, shape=(), dtype=float32)
tf.Tensor(1.0, shape=(), dtype=float32)


In [3]:
# Calculate the second order derivative

with tf.GradientTape() as tape2:
  with tf.GradientTape() as tape1:
    y = a * tf.pow(x, 2) + b * x + c

  dy_dx = tape1.gradient(y, x)

dy2_dx2 = tape2.gradient(dy_dx, x)
print(dy2_dx2)

tf.Tensor(2.0, shape=(), dtype=float32)


In [8]:
# Use it in the autograph

@tf.function
def f(x):
  a = tf.constant(1.0)
  b = tf.constant(-2.0)
  c = tf.constant(1.0)

  # Convert the type  of the variable to tf.float32
  x = tf.cast(x, tf.float32)

  with tf.GradientTape() as tape:
    tape.watch(x)
    y = a * tf.pow(x, 2) + b * x + c

  dy_dx = tape.gradient(y, x)
  return (dy_dx, x)

tf.print(f(tf.constant(0.0)))
tf.print(f(tf.constant(1.0)))

(-2, 0)
(0, 1)


## 2. Calculate the Minimal Value Through the Gradient Tape and the Optimizer

In [9]:
# Calculate the minimal value of f(x) = a*x**2 + b*x + c
# Use optimizer.apply_gradients

x = tf.Variable(0.0, name='x', dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for _ in range(1000):
  with tf.GradientTape() as tape:
    y = a * tf.pow(x, 2) + b * x + c

  dy_dx = tape.gradient(y, x)
  optimizer.apply_gradients(grads_and_vars=[(dy_dx, x)])

tf.print(f'y: {y}; x: {x}')

y: 0.0; x: <tf.Variable 'x:0' shape=() dtype=float32, numpy=0.99999857>


In [10]:
# Calculate the minimal value of f(x) = a*x**2 + b*x + c
# Use optimizer.minimize
# This optimizer.minimize is identical to calculating gradient using tape, then call apply_gradient

x = tf.Variable(0.0, name='x', dtype=tf.float32)

# f function has no argument
def f():
  a = tf.constant(1.0)
  b = tf.constant(-2.0)
  c = tf.constant(1.0)
  y = a * tf.pow(x, 2) + b * x + c
  return y

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for _ in range(1000):
  optimizer.minimize(f, var_list=[x])

tf.print(f'y: {y}; x: {x}')

y: 0.0; x: <tf.Variable 'x:0' shape=() dtype=float32, numpy=0.99999857>


In [11]:
# Calculate minimal value in autograph
# Use optimizer.apply_gradients

x = tf.Variable(0.0, name='x', dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

@tf.function
def minimize_f():
  a = tf.constant(1.0)
  b = tf.constant(-2.0)
  c = tf.constant(1.0)

  for _ in tf.range(1000): # Ise tf.range instead of range when using autograph
    with tf.GradientTape() as tape:
      y = a * tf.pow(x, 2) + b * x + c

    dy_dx = tape.gradient(y, x)
    optimizer.apply_gradients(grads_and_vars=[(dy_dx, x)])

  y = a * tf.pow(x, 2) + b * x + c
  return y

minimize_f()
tf.print(f'y: {y}; x: {x}')

y: 0.0; x: <tf.Variable 'x:0' shape=() dtype=float32, numpy=0.99999857>


In [13]:
# Calculate minimal value in autograph
# Use optimizer.minimize

x = tf.Variable(0.0, name='x', dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

@tf.function
def f():
  a = tf.constant(1.0)
  b = tf.constant(-2.0)
  c = tf.constant(1.0)
  y = a * tf.pow(x, 2) + b * x + c
  return (y)

@tf.function
def train(epoch):
  for _ in tf.range(epoch):
    optimizer.minimize(f, var_list=[x])

  return (f())

train(1000)
tf.print(f'y: {y}; x: {x}')

y: 0.0; x: <tf.Variable 'x:0' shape=() dtype=float32, numpy=0.99999857>
